# RAG sobre laptops + agente crítico


In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))


import pandas as pd
from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt", quiet=True)

from ej3_rag_agente import (
    load_and_normalize_dataset,
    build_chunks,
    build_bm25_index,
    retrieve_chunks,
    generate_answer_simple,
    critique_answer,
    answer_question
)

## Ingesta y normalizacion de datasets

Para mejorar la velocidad se tomo una muestra aleatoria de 200 filas y para la normalizacion no se realizaron grandes cambios, se recomienda para un futuro ampliar el paso de normalizacion.

In [2]:
df = load_and_normalize_dataset("../data/Laptops_with_technical_specifications.csv")
df.head()

,laptop_id,full_name,link_profile,producer,price_in_dollar,model,ram,disc,display_resolution,cpu,gpu
521,522,ASUS ROG Zephyrus G14,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS,NaN,ROGZephyrusG14,48GB,1000GB SSD,WQXGA (2560 x 1600),AMD Ryzen 7 7735HS,"NVIDIA GeForce RTX 4050 (Laptop, 120W)"
737,738,Lenovo IdeaPad Gaming 3i 15,https://laptopmedia.com/laptop-specs/lenovo-id...,Lenovo,$929.00,IdeaPadGaming3i15,16GB,1000GB SSD,Full HD (1920 x 1080),Intel Core i5-11300H,NVIDIA GeForce GTX 1650 (Laptop)
740,741,Lenovo ThinkPad E16 Gen 1,https://laptopmedia.com/laptop-specs/lenovo-th...,Lenovo,NaN,ThinkPadE16Gen1,8GB,2000GB SSD,WUXGA (1920 x 1200),Intel Core i5-1335U,Intel Iris Xe Graphics G7 (80EU)
660,661,ASUS TUF Gaming F17,https://laptopmedia.com/laptop-specs/asus-tuf-...,ASUS,NaN,TUFGamingF17,8GB,512GB SSD,Full HD (1920 x 1080),Intel Core i5-11260H,"NVIDIA GeForce RTX 3050 (Laptop, 75W)"
411,412,Acer Predator Helios 300,https://laptopmedia.com/laptop-specs/acer-pred...,Acer,NaN,PredatorHelios300,64GB,8000GB SSD,Full HD (1920 x 1080),Intel Core i7-12700H,"NVIDIA GeForce RTX 3060 (Laptop, 140W)"


## Construccion de chunks BM25

Cada laptop se divide en chunks por campo y se construye el indice sobre el texto de esos chunks

fields = [
        "producer",
        "full_name",
        "price_in_dollar",
        "model",
        "ram",
        "disc",
        "display_resolution",
        "cpu",
        "gpu",
    ]



In [3]:
chunks_texts, chunks_meta = build_chunks(df)

len(chunks_texts), chunks_texts[:10]

(1643,
 ['ASUS ROG Zephyrus G14 ASUS ROGZephyrusG14 producer: ASUS',
  'ASUS ROG Zephyrus G14 ASUS ROGZephyrusG14 full_name: ASUS ROG Zephyrus G14',
  'ASUS ROG Zephyrus G14 ASUS ROGZephyrusG14 model: ROGZephyrusG14',
  'ASUS ROG Zephyrus G14 ASUS ROGZephyrusG14 ram: 48GB',
  'ASUS ROG Zephyrus G14 ASUS ROGZephyrusG14 disc: 1000GB SSD',
  'ASUS ROG Zephyrus G14 ASUS ROGZephyrusG14 display_resolution:  WQXGA (2560 x 1600)',
  'ASUS ROG Zephyrus G14 ASUS ROGZephyrusG14 cpu: AMD Ryzen 7 7735HS',
  'ASUS ROG Zephyrus G14 ASUS ROGZephyrusG14 gpu: NVIDIA GeForce RTX 4050 (Laptop, 120W)',
  'Lenovo IdeaPad Gaming 3i 15 Lenovo IdeaPadGaming3i15 producer: Lenovo',
  'Lenovo IdeaPad Gaming 3i 15 Lenovo IdeaPadGaming3i15 full_name: Lenovo IdeaPad Gaming 3i 15'])

In [4]:
bm25, tokenized_corpus = build_bm25_index(chunks_texts)
len(tokenized_corpus)


1643

## Retrieve: Obtener los  k (k=5) chunks más relevantes 

Dada una query, se obtienen los k chunks más relevantes

In [5]:
query = "¿Cuánta memoria ram tiene Lenovo ThinkPad E16 Gen1?"
retrieved = retrieve_chunks(query, bm25, tokenized_corpus, chunks_meta)
retrieved

[{'laptop_id': 741,
  'field': 'full_name',
  'value': 'Lenovo ThinkPad E16 Gen 1',
  'text': 'Lenovo ThinkPad E16 Gen 1 Lenovo ThinkPadE16Gen1 full_name: Lenovo ThinkPad E16 Gen 1'},
 {'laptop_id': 880,
  'field': 'full_name',
  'value': 'Lenovo ThinkPad E16 Gen 1',
  'text': 'Lenovo ThinkPad E16 Gen 1 Lenovo ThinkPadE16Gen1 full_name: Lenovo ThinkPad E16 Gen 1'},
 {'laptop_id': 530,
  'field': 'full_name',
  'value': 'Lenovo ThinkPad E16 Gen 1',
  'text': 'Lenovo ThinkPad E16 Gen 1 Lenovo ThinkPadE16Gen1 full_name: Lenovo ThinkPad E16 Gen 1'},
 {'laptop_id': 73,
  'field': 'full_name',
  'value': 'Lenovo ThinkPad E16 Gen 1',
  'text': 'Lenovo ThinkPad E16 Gen 1 Lenovo ThinkPadE16Gen1 full_name: Lenovo ThinkPad E16 Gen 1'},
 {'laptop_id': 669,
  'field': 'full_name',
  'value': 'Lenovo ThinkPad E16 Gen 1',
  'text': 'Lenovo ThinkPad E16 Gen 1 Lenovo ThinkPadE16Gen1 full_name: Lenovo ThinkPad E16 Gen 1'}]

## Generate: Generar la respuesta

En esta etapa, el sistema toma la query del usuario y construye una respuesta concisa basada exclusivamente en la información recuperada desde los chunks.

Para este ejercicio se implementó un generador utilizando funciones auxiliares:

- guess_field_from_query: Identifica qué atributo pide el usuario (p. ej., RAM, CPU, GPU).
- choose_best_laptop_id: Selecciona el laptop más probable entre los chunks recuperados.
- format_citation: Formatea las citas con el esquema [laptop_id:campo].
- generate_answer_simple: Construye la frase final incluyendo el valor consultado y las citas correspondientes.

Nota: Para efectos de la prueba técnica, se utilizó un generador basado en reglas, ya que es más eficiente y fácil de auditar.

Sin embargo, en una versión futura podría reemplazarse por un modelo LLM.

In [6]:
answer_prelim = generate_answer_simple(query, retrieved, chunks_meta )
answer_prelim

'La Lenovo ThinkPad E16 Gen 1 tiene 8GB de ram [880:ram][880:full_name].'

## Agente critico: Verificar soporte de las afirmaciones

El agente critico divide las respuestas en oraciones y verifica que cada una tenga al menos una cita valida que provenga de los chunks recuperados.


In [7]:
critique = critique_answer(answer_prelim, retrieved)
critique

{'decision': 'accept',
 'supported': ['La Lenovo ThinkPad E16 Gen 1 tiene 8GB de ram [880:ram][880:full_name].'],
 'unsupported': []}

## Ciclo completo: Retrieve -> Generate -> Critique +logs

Esta función implementa el ciclo completo de inferencia para una pregunta:
1. Recupera chunks
2. Genera respuesrta preeliinar
3. El agente critico valida la respuesta
4. Si la respuesta no es valida, se vuelve a generar (hasta 2 intentos)
5. Registra cada intento en un archivo JSONL de logs.

In [8]:
answer = answer_question(query, bm25, tokenized_corpus, chunks_meta)
answer


'La Lenovo ThinkPad E16 Gen 1 tiene 8GB de ram [880:ram][880:full_name].'

In [9]:
test_queries = [
    "¿Cuánta memoria RAM tiene Lenovo ThinkPad E16 Gen 1?",
    "¿Cuanto cuesta la Lenovo IdeaPad Gaming 3i 15?",
    "¿vendes licuadoras?",
]

In [10]:
for q in test_queries:
    ans = answer_question(q, bm25, tokenized_corpus, chunks_meta)
    print("Pregunta:", q)
    print("Respuesta:", ans)
    print("-" * 80)

Pregunta: ¿Cuánta memoria RAM tiene Lenovo ThinkPad E16 Gen 1?
Respuesta: La Lenovo ThinkPad E16 Gen 1 tiene 8GB de ram [880:ram][880:full_name].
--------------------------------------------------------------------------------
Pregunta: ¿Cuanto cuesta la Lenovo IdeaPad Gaming 3i 15?
Respuesta: full_name: Lenovo IdeaPad Gaming 3i 15 [738:full_name] / full_name: Lenovo IdeaPad Gaming 3i 15 [140:full_name] / full_name: Lenovo IdeaPad Gaming 3i 15 [618:full_name]
--------------------------------------------------------------------------------
Pregunta: ¿vendes licuadoras?
Respuesta: producer: ASUS [522:producer] / full_name: ASUS ROG Zephyrus G14 [522:full_name] / model: ROGZephyrusG14 [522:model]
--------------------------------------------------------------------------------


En el caso de preguntas fuera de contexto (por ejemplo, “¿vendes licuadoras?”), el sistema sigue retornando información de alguna laptop, ya que no se implementó un clasificador de intención o detección explícita de consultas fuera de dominio, esto se reconoce como una limitación del prototipo actual.
